# K-Fold CV for Boosting Methods Using SKLearn 

In [1]:
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import wfdb
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
import random
from scipy.stats import randint

In [2]:
import xgboost as xgb
from sklearn import metrics
from lightgbm import LGBMClassifier

import lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

C:\Users\santo\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats

### To load sklearn dataset
sklearn.datasets.load_breast_cancer(*, return_X_y=False, as_frame=False)[source]¶

Load and return the breast cancer wisconsin dataset (classification).

In [4]:
#data = load_breast_cancer(as_frame = True); 
#df = data.frame; 
#print(df.shape)
# In this block use the code we looked at on 6/30 as a reference to save our rr-intervals as a .
rlist = []
records = 'mit-bih-raw\RECORDS' # Replace the _______ with the name of the records file in your MIT-BIH data folder
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it
        

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
  
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
        # Then replace _____ with the correct function call
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
            
        ######################################################
        samples.append(samp) #add it to our array for all subject
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list

        

  0%|          | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████| 25/25 [01:10<00:00,  2.81s/it]


In [5]:
features_df_list = []
for subject in good_list:
    data = pd.read_csv(f'subject-features-dataframes/subject{subject}.csv')
    data = data.assign(Subject = subject)
    #print(data)
    features_df_list.append(data)
#features_maindf = pd.concat(features_df_list, ignore_index = True)

In [6]:
#df.head(5)
# df.info()
# df.describe()
#features_df_list[0]
counter = 0
testing_arr = []
training_arr = []
for subject in features_df_list:
    if counter != 22:
        training_arr.append(subject)
    else:
        testing_arr.append(subject)
    counter += 1
    
print(len(training_arr))

22


In [6]:
features_maindf = pd.concat(features_df_list, ignore_index = True)
print(features_maindf.columns)

Index(['Unnamed: 0', 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation',
       'Rhythm', 'Subject'],
      dtype='object')


In [11]:
features_df_list[14]

,Unnamed: 0,StoS,StoR,StoL,RtoS,RtoR,RtoL,LtoS,LtoR,LtoL,...,Volatility,ShEn,AppEn,SampEn,IQR,Var,Rmean_Var,Mean_Absolute_Deviation,Rhythm,Subject
0,0,0.208333,0.041667,0.000000,0.000000,0.666667,0.041667,0.0,0.041667,0.0,...,0.004135,0.707786,0.286011,0.372675,0.080,0.003389,0.006766,0.048461,1,07859
1,1,0.083333,0.125000,0.000000,0.083333,0.541667,0.083333,0.0,0.083333,0.0,...,0.008079,0.760469,0.685069,0.763916,0.120,0.006420,0.005691,0.067917,1,07859
2,2,0.166667,0.083333,0.000000,0.041667,0.708333,0.000000,0.0,0.000000,0.0,...,0.006135,0.551080,0.405797,0.405465,0.072,0.004827,0.004312,0.054554,1,07859
3,3,0.125000,0.125000,0.000000,0.083333,0.666667,0.000000,0.0,0.000000,0.0,...,0.005843,0.551080,0.377900,0.414556,0.048,0.003657,0.008534,0.043840,1,07859
4,4,0.125000,0.041667,0.000000,0.000000,0.750000,0.041667,0.0,0.041667,0.0,...,0.003024,0.600483,0.148471,0.220241,0.044,0.002308,0.007991,0.035827,1,07859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2397,2397,0.125000,0.083333,0.000000,0.041667,0.666667,0.041667,0.0,0.041667,0.0,...,0.002978,0.659215,0.315960,0.400825,0.100,0.003282,0.007139,0.051802,1,07859
2398,2398,0.125000,0.083333,0.000000,0.041667,0.666667,0.041667,0.0,0.041667,0.0,...,0.003601,0.659215,0.228548,0.317358,0.064,0.003121,0.007762,0.045581,1,07859
2399,2399,0.125000,0.166667,0.000000,0.125000,0.583333,0.000000,0.0,0.000000,0.0,...,0.003471,0.592953,0.272772,0.339051,0.072,0.002899,0.008531,0.044288,1,07859
2400,2400,0.208333,0.083333,0.000000,0.041667,0.666667,0.000000,0.0,0.000000,0.0,...,0.005880,0.592953,0.338168,0.382369,0.092,0.004138,0.008303,0.053478,1,07859


# Extreme Gradient Boosting

In [13]:
# Short to Short



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['StoS']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9738059724209309


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.7813857832988267


...

roc_auc_score 0.8710001016880211


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9727491039426523


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9437591327375165
---Run time is 19.435571399999844 seconds ---

Accuracy of each fold: 
 [0.8483981693363845, 0.24178498985801217, 0.9861460957178841, 0.8920863309352518, 0.8359456635318704, 0.09776876267748479, 0.6202709014479215, 0.9904697072838666, 0.7399598393574297, 0.7276736493936052, 0.6311252268602541, 0.9849246231155779, 0.6063492063492063, 0.3159235668789809, 0.5645295587010825, 0.8817013735046522, 0.9129540781357094, 0.31217541834968265, 0.2569737954353339, 0.8290269378779549, 0.8387510692899914, 0.9648462021343377, 0.852431289640592]

Avg accuracy : 
0.6927050632962202

Std of accuracy : 
0.2669211118961723
[[15295  9448]
 [ 5524 14701]]

              precision    recall  f1-score   support

           0       0.73      0.62      0.67     24743
           1       0.61      0.73      0.66     20225

    accuracy                           0.67     44968
   macro avg       0.67      0.67      0.67     44968
weighted avg       0.68      0.67      0.67     44968

In [14]:
# Short to Regular



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['StoR']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9300325713068095


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9277768576029446


...

roc_auc_score 0.97397701850722


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9972508960573477


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.983586732780066
---Run time is 19.39458300000001 seconds ---

Accuracy of each fold: 
 [0.8604118993135011, 0.9294117647058824, 0.9861460957178841, 0.7685851318944844, 0.9848484848484849, 0.9046653144016227, 0.8734236338159739, 0.9891082368958475, 0.9126506024096386, 0.8930540242557883, 0.97005444646098, 0.9655419956927495, 0.5714285714285714, 0.9426751592356688, 0.8397169025811824, 0.9109437306158618, 0.980123372172721, 0.9249855741488748, 0.5832628909551987, 0.9417262231995602, 0.9666381522668948, 0.9918392969240427, 0.9733615221987315]

Avg accuracy : 
0.898461001136528

Std of accuracy : 
0.1129152772168261
[[21187  3556]
 [ 1329 18896]]

              precision    recall  f1-score   support

           0       0.94      0.86      0.90     24743
           1       0.84      0.93      0.89     20225

    accuracy                           0.89     44968
   macro avg       0.89      0.90      0.89     44968
weighted avg       0.90      0.89      0.89     44968


[0.86

In [15]:
# Short to Long



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['StoL']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.5538644181303461


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.7643487462617897


...

roc_auc_score 0.835026947325605


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.643451612903226


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.7778997549673694
---Run time is 19.79956909999987 seconds ---

Accuracy of each fold: 
 [0.8907322654462243, 0.860446247464503, 0.9219143576826196, 0.710431654676259, 0.754440961337513, 0.8677484787018256, 0.6795889771134984, 0.94213750850919, 0.5737951807228916, 0.6306504961411246, 0.7078039927404719, 0.7753050969131371, 0.6621315192743764, 0.7490445859872611, 0.2402164862614488, 0.6566238369517058, 0.7786154900616861, 0.6237738026543566, 0.39391377852916315, 0.8053875755909841, 0.7869974337040205, 0.8606403013182674, 0.7128964059196617]

Avg accuracy : 
0.7210972362479212

Std of accuracy : 
0.15952468236362016
[[20087  4656]
 [ 8430 11795]]

              precision    recall  f1-score   support

           0       0.70      0.81      0.75     24743
           1       0.72      0.58      0.64     20225

    accuracy                           0.71     44968
   macro avg       0.71      0.70      0.70     44968
weighted avg       0.71      0.71      0.70     44968


[0.

In [16]:
# Regular to Short



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['RtoS']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.8930930565279596


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9442019783758914


...

roc_auc_score 0.9697620500305064


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9098960573476702


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9808084210604893
---Run time is 19.313732000000073 seconds ---

Accuracy of each fold: 
 [0.8689931350114416, 0.9115618661257606, 0.9836272040302267, 0.6810551558752997, 0.9409613375130617, 0.9087221095334685, 0.8397944885567492, 0.9782164737916951, 0.8664658634538153, 0.705071664829107, 0.9428312159709619, 0.9339554917444365, 0.6281179138321995, 0.9305732484076433, 0.5603663613655288, 0.8152414709791759, 0.9225496915695682, 0.9203693017888055, 0.4974640743871513, 0.9092908191313909, 0.916595380667237, 0.8807281858129316, 0.9035940803382664]

Avg accuracy : 
0.8454846319441703

Std of accuracy : 
0.1322663495924011
[[20586  4157]
 [ 3200 17025]]

              precision    recall  f1-score   support

           0       0.87      0.83      0.85     24743
           1       0.80      0.84      0.82     20225

    accuracy                           0.84     44968
   macro avg       0.83      0.84      0.84     44968
weighted avg       0.84      0.84      0.84     44968


[

In [17]:
# Regular to Regular



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['RtoR']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9505646707862774


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9769845870715435


...

roc_auc_score 0.9949425462680497


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9491648745519713


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9994387390430477
---Run time is 20.85483249999993 seconds ---

Accuracy of each fold: 
 [0.8918764302059496, 0.9391480730223124, 0.993073047858942, 0.8165467625899281, 0.9848484848484849, 0.9079107505070994, 0.8748248482017749, 0.9625595643294759, 0.9061244979919679, 0.7502756339581036, 0.9777676950998185, 0.9425699928212491, 0.7519274376417233, 0.9796178343949045, 0.45087427144046627, 0.972972972972973, 0.9492803289924606, 0.93133294864397, 0.5287404902789519, 0.936778449697636, 0.9786142001710864, 0.9070935342121783, 0.9881606765327695]

Avg accuracy : 
0.8836051707136621

Std of accuracy : 
0.13910748456569083
[[21121  3622]
 [ 1990 18235]]

              precision    recall  f1-score   support

           0       0.91      0.85      0.88     24743
           1       0.83      0.90      0.87     20225

    accuracy                           0.88     44968
   macro avg       0.87      0.88      0.87     44968
weighted avg       0.88      0.88      0.88     44968


[0.

In [18]:
# Regular to Long



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['RtoL']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9140513226385801


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.903384403036577


...

roc_auc_score 0.9597681513117754


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9693978494623656


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9612151416355789
---Run time is 16.44497720000004 seconds ---

Accuracy of each fold: 
 [0.8918764302059496, 0.9334685598377281, 0.9716624685138538, 0.8962829736211031, 0.9378265412748171, 0.954158215010142, 0.7949556282111163, 0.9639210347174949, 0.8744979919678715, 0.9388092613009923, 0.9224137931034483, 0.955491744436468, 0.8403628117913832, 0.943312101910828, 0.6144879267277269, 0.8985378821444395, 0.9561343385880741, 0.9388343912290825, 0.8423499577345731, 0.9477735019241341, 0.9396920444824637, 0.9824231010671689, 0.9424947145877378]

Avg accuracy : 
0.9079029310603738

Std of accuracy : 
0.07732287419659935
[[23153  1590]
 [ 2838 17387]]

              precision    recall  f1-score   support

           0       0.89      0.94      0.91     24743
           1       0.92      0.86      0.89     20225

    accuracy                           0.90     44968
   macro avg       0.90      0.90      0.90     44968
weighted avg       0.90      0.90      0.90     44968


[0

In [19]:
# Long to Short



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['LtoS']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.6183525615658579


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.7245244996549344


...

roc_auc_score 0.8579789505796217


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.7232795698924732


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.8406242826777048
---Run time is 19.30106550000005 seconds ---

Accuracy of each fold: 
 [0.9136155606407322, 0.8543610547667343, 0.9848866498740554, 0.8890887290167866, 0.7826541274817137, 0.9379310344827586, 0.6562354040168146, 0.9863852961198094, 0.6154618473895582, 0.8798235942668137, 0.6710526315789473, 0.9454414931801867, 0.672108843537415, 0.7630573248407644, 0.14321398834304747, 0.6517501107665042, 0.9198080877313228, 0.6508944027697634, 0.8169907016060862, 0.8317757009345794, 0.7874251497005988, 0.9365976145637163, 0.7657505285412262]

Avg accuracy : 
0.7850569511369535

Std of accuracy : 
0.17873100908345882
[[23610  1133]
 [ 9248 10977]]

              precision    recall  f1-score   support

           0       0.72      0.95      0.82     24743
           1       0.91      0.54      0.68     20225

    accuracy                           0.77     44968
   macro avg       0.81      0.75      0.75     44968
weighted avg       0.80      0.77      0.76     44968



In [20]:
# Long to Regular



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['LtoR']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9098809777480137


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9027568438003221


...

roc_auc_score 0.9309553589587146


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.8958028673835126


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.91544694660976
---Run time is 18.777681899999834 seconds ---

Accuracy of each fold: 
 [0.8712814645308925, 0.9176470588235294, 0.9074307304785895, 0.6786570743405276, 0.8766980146290491, 0.8945233265720081, 0.8187762727697337, 0.9196732471068755, 0.7911646586345381, 0.6119073869900772, 0.9065335753176044, 0.7609475951184493, 0.7705215419501134, 0.9363057324840764, 0.6369691923397169, 0.9091714665485158, 0.7950651130911583, 0.8926716676283901, 0.46956889264581575, 0.9054425508521166, 0.9067579127459366, 0.8593848085373509, 0.8862579281183932]

Avg accuracy : 
0.8227546614023242

Std of accuracy : 
0.11801373500160083
[[19563  5180]
 [ 2898 17327]]

              precision    recall  f1-score   support

           0       0.87      0.79      0.83     24743
           1       0.77      0.86      0.81     20225

    accuracy                           0.82     44968
   macro avg       0.82      0.82      0.82     44968
weighted avg       0.83      0.82      0.82     44968



In [21]:
# Long to Long



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['LtoL']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.6510304100331801


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.5950393374741201


...

roc_auc_score 0.7369396990034573


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.608258064516129


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.6999600632270032
---Run time is 16.31611440000006 seconds ---

Accuracy of each fold: 
 [0.9262013729977117, 0.8020283975659229, 0.9842569269521411, 0.8099520383693045, 0.5501567398119123, 0.9156186612576065, 0.5035030359645025, 0.9768550034036759, 0.4643574297188755, 0.9454244762954797, 0.39655172413793105, 0.9842067480258435, 0.6408163265306123, 0.47579617834394905, 0.02289758534554538, 0.461674789543642, 0.8773132282385195, 0.5164454702827467, 0.7852916314454776, 0.7927432655305112, 0.5988023952095808, 0.9353421217827997, 0.5577167019027485]

Avg accuracy : 
0.6923457499416105

Std of accuracy : 
0.2433333867674072
[[23951   792]
 [14017  6208]]

              precision    recall  f1-score   support

           0       0.63      0.97      0.76     24743
           1       0.89      0.31      0.46     20225

    accuracy                           0.67     44968
   macro avg       0.76      0.64      0.61     44968
weighted avg       0.75      0.67      0.63     44968


In [22]:
# Standard Deviation



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['STD']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.8756811055980032


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9560961582700713


...

roc_auc_score 0.9717505592841164


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.8879175627240143


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9750180555270441
---Run time is 55.425044899999875 seconds ---

Accuracy of each fold: 
 [0.8060640732265446, 0.9233265720081136, 0.8828715365239295, 0.41786570743405277, 0.9158829676071055, 0.8997971602434077, 0.9117234936945353, 0.8740639891082369, 0.8524096385542169, 0.5705622932745315, 0.9745916515426497, 0.7315147164393395, 0.6222222222222222, 0.8273885350318472, 0.21815154038301415, 0.8763845813026141, 0.798492117888965, 0.9365262550490479, 0.43956043956043955, 0.8911489829576691, 0.9503849443969205, 0.8279974890144381, 0.9454545454545454]

Avg accuracy : 
0.7867124109964516

Std of accuracy : 
0.19495337570645044
[[17663  7080]
 [ 2598 17627]]

              precision    recall  f1-score   support

           0       0.87      0.71      0.78     24743
           1       0.71      0.87      0.78     20225

    accuracy                           0.78     44968
   macro avg       0.79      0.79      0.78     44968
weighted avg       0.80      0.78      0.78     4496

In [23]:
# Coefficient of Variance



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['CoefVar']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9190740007914523


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9693604784909132


...

roc_auc_score 0.9830633516371771


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9211362007168459


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9947009127605454
---Run time is 63.722636800000146 seconds ---

Accuracy of each fold: 
 [0.8546910755148741, 0.9318458417849899, 0.9672544080604534, 0.685251798561151, 0.9717868338557993, 0.9107505070993914, 0.9159271368519384, 0.9121851599727706, 0.8995983935742972, 0.6945975744211687, 0.9754990925589837, 0.873653984206748, 0.8095238095238095, 0.9987261146496815, 0.22231473771856786, 0.9304386353566682, 0.9163810829335161, 0.9572994806693594, 0.5540997464074388, 0.8949972512369434, 0.9674935842600513, 0.871939736346516, 0.9742071881606765]

Avg accuracy : 
0.856107094509817

Std of accuracy : 
0.17141873211366715
[[20359  4384]
 [ 2412 17813]]

              precision    recall  f1-score   support

           0       0.89      0.82      0.86     24743
           1       0.80      0.88      0.84     20225

    accuracy                           0.85     44968
   macro avg       0.85      0.85      0.85     44968
weighted avg       0.85      0.85      0.85     44968


[

In [24]:
# Range



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['Range']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.8339167757450306


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9568083735909824


...

roc_auc_score 0.9190842993695342


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.8851971326164874


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9350299619107693
---Run time is 23.691810599999826 seconds ---

Accuracy of each fold: 
 [0.7757437070938215, 0.9119675456389452, 0.853904282115869, 0.30755395683453235, 0.9184952978056427, 0.8762677484787018, 0.9173283512377394, 0.8495575221238938, 0.8418674698795181, 0.5369349503858876, 0.9727767695099818, 0.6999282124910265, 0.5551020408163265, 0.9503184713375796, 0.42631140716069943, 0.8768276473194506, 0.7930089102124743, 0.93133294864397, 0.37193575655114114, 0.8807036833424958, 0.9525235243798118, 0.8173258003766478, 0.9441860465116279]

Avg accuracy : 
0.7809522630542515

Std of accuracy : 
0.1952267362310946
[[16648  8095]
 [ 1766 18459]]

              precision    recall  f1-score   support

           0       0.90      0.67      0.77     24743
           1       0.70      0.91      0.79     20225

    accuracy                           0.78     44968
   macro avg       0.80      0.79      0.78     44968
weighted avg       0.81      0.78      0.78     44968



In [25]:
# Volatility



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['Volatility']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.8261087942528386


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9580064412238326


...

roc_auc_score 0.9073306894447835


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.8551720430107527


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9118675666748156
---Run time is 38.11438590000034 seconds ---

Accuracy of each fold: 
 [0.7780320366132724, 0.9103448275862069, 0.843198992443325, 0.2949640287769784, 0.912748171368861, 0.8758620689655172, 0.9098552078468005, 0.853641933287951, 0.8383534136546185, 0.5347298787210585, 0.9709618874773139, 0.6992103374012921, 0.5519274376417234, 0.9834394904458599, 0.47751873438801, 0.9100575985821887, 0.794379712131597, 0.9244085401038661, 0.36601859678782755, 0.8763056624518967, 0.9461077844311377, 0.8148148148148148, 0.9374207188160677]

Avg accuracy : 
0.7827957336842688

Std of accuracy : 
0.19435922819782642
[[16579  8164]
 [ 1597 18628]]

              precision    recall  f1-score   support

           0       0.91      0.67      0.77     24743
           1       0.70      0.92      0.79     20225

    accuracy                           0.78     44968
   macro avg       0.80      0.80      0.78     44968
weighted avg       0.81      0.78      0.78     44968


[0.7

In [26]:
# Shannon Entropy



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['ShEn']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9601381997503882


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9738334483551876


...

roc_auc_score 0.9861836485661989


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9629856630824373


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9983339460704268
---Run time is 11.910300200000165 seconds ---

Accuracy of each fold: 
 [0.8975972540045767, 0.9257606490872211, 0.9911838790931989, 0.8165467625899281, 0.9827586206896551, 0.8855983772819472, 0.8229799159271368, 0.9816201497617427, 0.8975903614457831, 0.737045203969129, 0.9764065335753176, 0.9547738693467337, 0.7854875283446712, 0.9490445859872612, 0.5295587010824313, 0.9725299069561365, 0.9465387251542152, 0.860934795152914, 0.4661876584953508, 0.8867509620670698, 0.9854576561163387, 0.9146264908976773, 0.9894291754756871]

Avg accuracy : 
0.8763655548913968

Std of accuracy : 
0.13573156735172223
[[20733  4010]
 [ 1917 18308]]

              precision    recall  f1-score   support

           0       0.92      0.84      0.87     24743
           1       0.82      0.91      0.86     20225

    accuracy                           0.87     44968
   macro avg       0.87      0.87      0.87     44968
weighted avg       0.87      0.87      0.87     44968




In [27]:
# Approximate Entropy



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['AppEn']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9177346199506864


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9762447665056361


...

roc_auc_score 0.9973754321740899


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9855125448028673


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9987561748036287
---Run time is 28.052977700000156 seconds ---

Accuracy of each fold: 
 [0.8838672768878718, 0.9379310344827586, 0.9937027707808564, 0.6354916067146283, 0.9796238244514106, 0.9221095334685598, 0.8902382064455862, 0.988427501701838, 0.9252008032128514, 0.8787210584343991, 0.9800362976406534, 0.9633883704235463, 0.7315192743764173, 1.0, 0.2922564529558701, 0.844040762073549, 0.976010966415353, 0.9971148297749567, 0.6191885038038885, 0.9472237493128093, 0.9666381522668948, 0.935969868173258, 0.9446088794926004]

Avg accuracy : 
0.8797091184039374

Std of accuracy : 
0.16313739372799654
[[21551  3192]
 [ 2721 17504]]

              precision    recall  f1-score   support

           0       0.89      0.87      0.88     24743
           1       0.85      0.87      0.86     20225

    accuracy                           0.87     44968
   macro avg       0.87      0.87      0.87     44968
weighted avg       0.87      0.87      0.87     44968


[0.88386727688787

In [28]:
# Sample Entropy



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['SampEn']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9243097622599008


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9776590752242925


...

roc_auc_score 0.9968385194224121


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9783906810035842


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9993360978040373
---Run time is 18.3627677999998 seconds ---

Accuracy of each fold: 
 [0.8832951945080092, 0.9460446247464503, 0.993073047858942, 0.6199040767386091, 0.9817136886102403, 0.922920892494929, 0.9065857076132648, 0.9891082368958475, 0.9272088353413654, 0.8158765159867696, 0.9823049001814882, 0.9569274946159368, 0.7034013605442176, 1.0, 0.37635303913405493, 0.8852459016393442, 0.9677861549006168, 0.9976918638199653, 0.5718512256973796, 0.9538207806487081, 0.9820359281437125, 0.9146264908976773, 0.9682875264270613]

Avg accuracy : 
0.8802636298888953

Std of accuracy : 
0.15783498513557123
[[21115  3628]
 [ 2185 18040]]

              precision    recall  f1-score   support

           0       0.91      0.85      0.88     24743
           1       0.83      0.89      0.86     20225

    accuracy                           0.87     44968
   macro avg       0.87      0.87      0.87     44968
weighted avg       0.87      0.87      0.87     44968


[0.8832951945080

In [29]:
# Interquartile Range



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['IQR']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9428480107150468


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.959757533931447


...

roc_auc_score 0.9956391092129347


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9858888888888889


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9986106108646684
---Run time is 14.186191400000098 seconds ---

Accuracy of each fold: 
 [0.9061784897025171, 0.9269776876267748, 0.9817380352644837, 0.7200239808153477, 0.9205851619644723, 0.9598377281947261, 0.8916394208313873, 0.9931926480599047, 0.9191767068273092, 0.8776185226019846, 0.9786751361161524, 0.9877961234745154, 0.8149659863945579, 0.8993630573248408, 0.7497918401332223, 0.8834736375719983, 0.9876627827278959, 1.0, 0.7235841081994928, 0.94282572842221, 0.9769033361847733, 0.9139987445072191, 0.9826638477801268]

Avg accuracy : 
0.9103770743793874

Std of accuracy : 
0.08302408698676281
[[22042  2701]
 [ 1517 18708]]

              precision    recall  f1-score   support

           0       0.94      0.89      0.91     24743
           1       0.87      0.92      0.90     20225

    accuracy                           0.91     44968
   macro avg       0.90      0.91      0.91     44968
weighted avg       0.91      0.91      0.91     44968


[0.906178489702

In [30]:
# Variance



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['Var']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.8756811055980032


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9560961582700713


...

roc_auc_score 0.9717505592841164


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.8879175627240143


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9750180555270441
---Run time is 37.62199439999995 seconds ---

Accuracy of each fold: 
 [0.8060640732265446, 0.9233265720081136, 0.8828715365239295, 0.41786570743405277, 0.9158829676071055, 0.8997971602434077, 0.9117234936945353, 0.8740639891082369, 0.8524096385542169, 0.5705622932745315, 0.9745916515426497, 0.7315147164393395, 0.6222222222222222, 0.8273885350318472, 0.21815154038301415, 0.8763845813026141, 0.798492117888965, 0.9365262550490479, 0.43956043956043955, 0.8911489829576691, 0.9503849443969205, 0.8279974890144381, 0.9454545454545454]

Avg accuracy : 
0.7867124109964516

Std of accuracy : 
0.19495337570645044
[[17663  7080]
 [ 2598 17627]]

              precision    recall  f1-score   support

           0       0.87      0.71      0.78     24743
           1       0.71      0.87      0.78     20225

    accuracy                           0.78     44968
   macro avg       0.79      0.79      0.78     44968
weighted avg       0.80      0.78      0.78     44968

In [31]:
# Rmean Variance



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['Rmean_Var']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9735472283948738


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.8456829997699562


...

roc_auc_score 0.9597712019524101


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9689068100358422


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9633108891157365
---Run time is 50.07041580000032 seconds ---

Accuracy of each fold: 
 [0.8872997711670481, 0.23569979716024342, 0.9767002518891688, 0.8057553956834532, 0.8819226750261233, 0.0949290060851927, 0.7239607659971976, 0.9523485364193329, 0.7916666666666666, 0.7833517089305403, 0.7740471869328494, 0.9669777458722182, 0.6566893424036281, 0.6350318471337579, 0.6049125728559533, 0.9277802392556491, 0.9348869088416724, 0.5522215810732833, 0.25570583262890956, 0.8301264431006047, 0.897775876817793, 0.9322033898305084, 0.9052854122621564]

Avg accuracy : 
0.7394469110449543

Std of accuracy : 
0.242426437308116
[[15693  9050]
 [ 3816 16409]]

              precision    recall  f1-score   support

           0       0.80      0.63      0.71     24743
           1       0.64      0.81      0.72     20225

    accuracy                           0.71     44968
   macro avg       0.72      0.72      0.71     44968
weighted avg       0.73      0.71      0.71     44968




In [32]:
# Mean Absolute Deviation



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
x_values = ['Mean_Absolute_Deviation']
y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
#     param_distributions = {'n_estimators': randint(105, 110),
#                            'max_depth': randint(7,10)}
#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])    

    #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
    model.fit(X_train, y_train)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
#     plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    try:
        print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    except:
        print(f'ROC_AUC not able to be calculated for subject {subject}')
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

1.5.1


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.898739764390734


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9587200368069932


...

roc_auc_score 0.9889841366687004


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9361863799283154


C:\Users\santo\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


roc_auc_score 0.9962993167825891
---Run time is 31.888118100000156 seconds ---

Accuracy of each fold: 
 [0.8489702517162472, 0.9294117647058824, 0.9559193954659949, 0.5149880095923262, 0.9294670846394985, 0.9148073022312373, 0.9065857076132648, 0.9244383934649422, 0.9041164658634538, 0.6923925027563396, 0.9745916515426497, 0.8722182340272793, 0.6934240362811791, 0.8057324840764332, 0.23688592839300582, 0.8936641559592379, 0.9013022618231665, 0.9809578765147143, 0.547759932375317, 0.9230346344145135, 0.9709153122326775, 0.871939736346516, 0.9708245243128965]

Avg accuracy : 
0.8332325063629902

Std of accuracy : 
0.17898175589129453
[[19638  5105]
 [ 2633 17592]]

              precision    recall  f1-score   support

           0       0.88      0.79      0.84     24743
           1       0.78      0.87      0.82     20225

    accuracy                           0.83     44968
   macro avg       0.83      0.83      0.83     44968
weighted avg       0.83      0.83      0.83     44968

